In [6]:
%load_ext autoreload
%autoreload 2


import numpy as np
import pandas as pd
import anndata as ad
import numpy as np
import pandas as pd
import jax.numpy as jnp
import jax
import cellflow
jax.config.update("jax_default_device", jax.devices()[0])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
adata  = ad.read_h5ad('pbmc_cytokines_train.h5ad')

In [8]:
adata

AnnData object with n_obs × n_vars = 9593880 × 2000
    obs: 'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'bc1_wind', 'bc2_wind', 'bc3_wind', 'bc1_well', 'bc2_well', 'bc3_well', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'total_counts_MT', 'pct_counts_MT', 'log1p_total_counts_MT', 'donor', 'cytokine', 'treatment', 'cell_type', 'cytokine_family', 'condition', 'is_control'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'donor_embeddings', 'esm2_embeddings', 'hvg', 'log1p', 'pca'
    obsm: 'X_pca'
    varm: 'PCs', 'X_mean'

In [14]:
cf = cellflow.model.CellFlow(
    adata=adata,
    solver='otfm'
)

In [15]:
cf.prepare_data(
    sample_rep = "X_pca",
    control_key = "is_control",
    perturbation_covariates = {"cytokine_treatment": ("cytokine",)},
    perturbation_covariate_reps = {"cytokine_treatment": "esm2_embeddings"},
    sample_covariates = ["donor"],
    sample_covariate_reps = {"donor": "donor_embeddings"},
    split_covariates = ["donor"],
    max_combination_length = 1,
    null_value = 0.0,
)

[########################################] | 100% Completed | 3.42 ss


/ictstr01/home/icb/selman.ozleyen/projects/cell_flow_perturbation/src/cellflow/data/_datamanager.py:753: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in asarray is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  split_covariates_mask = jnp.asarray(df["split_covariates_mask"].values, dtype=jnp.int32)
/home/icb/selman.ozleyen/miniforge3/envs/cellflow/lib/python3.11/site-packages/jax/_src/numpy/lax_numpy.py:5540: RuntimeWarning: invalid value encountered in cast
  out = np.asarray(object, dtype=dtype)
/ictstr01/home/icb/selman.ozleyen/projects/cell_flow_perturbation/src/cellflow/data/_datamanager.py:754: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in asarray is not available, and will be truncated to dtype int32. To enable more dtypes, set the ja

[########################################] | 100% Completed | 1.19 sms


In [16]:
split_cov_combs = cf.data_manager._get_split_cov_combs(covariate_data=cf.data_manager.adata.obs)
o = cf.data_manager._get_condition_data_old(
    adata=cf.data_manager.adata,
    split_cov_combs=split_cov_combs,
)

/ictstr01/home/icb/selman.ozleyen/projects/cell_flow_perturbation/src/cellflow/data/_datamanager.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _covariate_data["cell_index"] = _covariate_data.index
 86%|████████▌ | 77/90 [00:35<00:05,  2.36it/s]

In [ ]:
n = cf.data_manager._get_condition_data(
    adata=cf.data_manager.adata,
    split_cov_combs=split_cov_combs,
)

[########################################] | 100% Completed | 4.03 ss


/ictstr01/home/icb/selman.ozleyen/projects/cell_flow_perturbation/src/cellflow/data/_datamanager.py:753: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in asarray is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  split_covariates_mask = jnp.asarray(df["split_covariates_mask"].values, dtype=jnp.int32)
/home/icb/selman.ozleyen/miniforge3/envs/cellflow/lib/python3.11/site-packages/jax/_src/numpy/lax_numpy.py:5540: RuntimeWarning: invalid value encountered in cast
  out = np.asarray(object, dtype=dtype)
/ictstr01/home/icb/selman.ozleyen/projects/cell_flow_perturbation/src/cellflow/data/_datamanager.py:754: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in asarray is not available, and will be truncated to dtype int32. To enable more dtypes, set the ja

[########################################] | 100% Completed | 862.27 ms


In [ ]:
attrs=[
    "split_idx_to_covariates",
    "perturbation_idx_to_covariates",
    "perturbation_idx_to_id",
    "condition_data",  # type: ignore[arg-type]
    "control_to_perturbation",
    "split_covariates_mask",
    "perturbation_covariates_mask",
]
for attr in attrs:
    old_attr = getattr(o, attr)
    new_attr = getattr(n, attr)
    if attr  in {"split_idx_to_covariates"}:
        assert set(old_attr.keys()) == set(new_attr.keys()), f"Mismatch in {attr} keys, len old: {len(old_attr)}, len new: {len(new_attr)}, old: {old_attr.keys()}, new: {new_attr.keys()}"
        for key in old_attr.keys():
            assert new_attr[key] == old_attr[key], f"Mismatch in {attr} for key {key}, old: {old_attr[key]}, new: {new_attr[key]}"

    elif isinstance(old_attr, dict):
        assert set(old_attr.keys()) == set(new_attr.keys()), f"Mismatch in {attr} keys"
        for key in old_attr.keys():
            old_ = np.array(old_attr[key])
            new_ = np.array(new_attr[key])
            assert np.array_equal(old_, new_), f"Mismatch in {attr} for key {key}, old: {old_}, new: {new_}"
    else:
        np_old = np.array(old_attr)
        np_new = np.array(new_attr)
        print(np_old, np_new)
        assert np.array_equal(np_old, np_new), f"Mismatch in {attr} values"

    print(f"Passed {attr} check")

AssertionError: Mismatch in split_idx_to_covariates keys, len old: 12, len new: 9, old: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]), new: dict_keys([0, 4, 5, 6, 7, 8, 9, 10, 11])

In [9]:
o.split_idx_to_covariates

{0: ('Donor1',),
 1: ('Donor2',),
 2: ('Donor3',),
 3: ('Donor4',),
 4: ('Donor5',),
 5: ('Donor6',),
 6: ('Donor7',),
 7: ('Donor8',),
 8: ('Donor9',),
 9: ('Donor10',),
 10: ('Donor11',),
 11: ('Donor12',)}